## Example script illustrating sound classification on audio stream
This notebook is an example of how to use DeGirum PySDK to do sound classification AI inference of an audio stream.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option.

You also need to specify your cloud API access token in [env.ini](../../env.ini) file, located in the same repository as this notebook.

**pyaudio package with portaudio is required to run this sample**

The script uses a WAV file for inference. Alternatively, you may use local microphone connected to the machine, by changing the `audio_source`.
The mic index or WAV filename needs to be specified either in the code below by assigning `audio_source`.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

# to install pyaudio package, uncomment the following lines
#!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0
#!pip show pyaudio || pip install pyaudio

#### Specify where do you want to run your inferences, model_zoo_url, model_name, and audio_source

In [ ]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# model_name: name of the model for running AI inference
# audio_source: audio source for inference
#     microphone index for local microphone
#     path to audio file (mp4/wav etc)
hw_location = "@cloud"
model_zoo_url = "https://cs.degirum.com/degirum/public"
model_name = "mobilenet_v1_yamnet_sound_cls--96x64_quant_n2x_orca1_1"
audio_source = "../../images/example_audio.wav"

#### The rest of the cells below should run without any modifications

In [ ]:
import sys
import numpy as np
from IPython.display import clear_output
import degirum as dg, degirum_tools

# configure for Google Colab
degirum_tools.configure_colab()
# connect to AI inference engine getting token from env.ini file
zoo = dg.connect(hw_location, model_zoo_url, degirum_tools.get_token())
# load sound classification model
model = zoo.load_model(model_name)

abort = False  # stream abort flag
N = 5  # inference results history depth
history = []  # list of N consecutive inference results

sampling_rate_hz = model.model_info.InputSamplingRate[0]
read_buf_size = model.model_info.InputWaveformSize[0] // 2  # to have 50% overlap

# Acquire model input stream object
with degirum_tools.open_audio_stream(
    sampling_rate_hz, read_buf_size, audio_source
) as stream:
    #
    # AI prediction loop.
    # emit keyboard typing sound to stop
    #
    for res in model.predict_batch(
        degirum_tools.audio_overlapped_source(stream, lambda: abort)
    ):
        # add top inference result to history
        history.insert(0, f"{res.results[0]['label']}: {res.results[0]['score']}")
        if len(history) > N:  # keep only N last elements in history
            history.pop()

        clear_output(wait=True)  # clear Jupyter output cell
        for m in history:  # print history
            print(m)

        if res.results[0]["label"] == "Typing":  # check for stop condition
            abort = True